In [1]:
library(tidyverse)
library(Seurat)

source('milo_cellbender.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [2]:
other_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_other_01') %>% 
    set_labels_to_labels_chunk %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
other_clusters

[1] "g1"     "g3"     "g2"     "g4"     "g_drop" "g7"

In [3]:
neuron_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_neuron_00') %>% 
    set_labels_to_labels_chunk %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
neuron_clusters 

[1] "n1"     "n0"     "n_drop" "n6"     "n4"     "n2"     "n8"     "n5"    
[9] "n3"

In [4]:
day_splits = tibble(object = rlang::syms(c("exp_labelled_other", "exp_labelled_neuron")))
# days = tibble(day = c("Day5", "Day14", "all"))
days = tibble(day = c("Day5", "Day14"))
day_splits = tidyr::crossing(day_splits, days)
day_splits

object,day
<list>,<chr>
exp_labelled_other,Day14
exp_labelled_other,Day5
exp_labelled_neuron,Day14
exp_labelled_neuron,Day5


In [5]:
other_clusters_tibble = tibble(label = other_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'other'))


neuron_clusters_tibble = tibble(label = neuron_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'neuron'))

clusters_tibble = rbind(other_clusters_tibble, neuron_clusters_tibble)
clusters_tibble = clusters_tibble %>% 
mutate(cluster = label) %>%
rowwise %>%
mutate(label = case_when(day != "all" ~ paste0(c(day, as.character(label)), collapse='.'),
                          TRUE ~ paste0(label, collapse='.'))) %>%
filter(!str_detect(cluster, 'unassigned'))
clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<chr>
Day14.g_drop,exp_labelled_other,Day14,g_drop
Day5.g_drop,exp_labelled_other,Day5,g_drop
Day14.g1,exp_labelled_other,Day14,g1
Day5.g1,exp_labelled_other,Day5,g1
Day14.g2,exp_labelled_other,Day14,g2
Day5.g2,exp_labelled_other,Day5,g2
Day14.g3,exp_labelled_other,Day14,g3
Day5.g3,exp_labelled_other,Day5,g3
Day14.g4,exp_labelled_other,Day14,g4


In [6]:
qs::qsave(clusters_tibble, 'clusters_tibble_labels_chunk.qs')